In [1]:
# Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import pandas as pd
# Set pandas options to not truncate output
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
import numpy as np
import json
import os
import sys
import datetime

2024-10-23 15:29:47.577585: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-23 15:29:47.593741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 15:29:47.612778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 15:29:47.618474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 15:29:47.631581: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Get the root directory of the system
# Get the virtual environment path
virtual_env_path = 'teamspace/uploads'

# Get the parent directory (the folder that contains the virtual env folder)
parent_directory = os.path.dirname(virtual_env_path)

print(f"Virtual environment directory: {virtual_env_path}")
print(f"Parent directory of the virtual environment folder: {parent_directory}")

Virtual environment directory: teamspace/uploads
Parent directory of the virtual environment folder: teamspace


In [3]:
# for timing purpose
print(datetime.datetime.now())

# turn off pandas Setting with Copy Warning
pd.set_option("mode.chained_assignment", None)

# read sample data
data_file_path = '/teamspace/uploads/hackathon_sample_v3.csv' 
factor_char_path = os.path.join(parent_directory, r"/teamspace/uploads/factor_char_list.csv")  

raw = pd.read_csv(data_file_path, parse_dates=["date"], low_memory=False)  # the date is the first day of the return month (t+1)

stock_vars = list(pd.read_csv(factor_char_path)["variable"].values)



2024-10-23 15:32:14.929787


In [7]:
df = pd.DataFrame(raw)
df.head()



,Unnamed: 0,date,ret_eom,permno,SHRCD,EXCHCD,mspread,year,month,RF,stock_exret,stock_ticker,CUSIP,comp_name,eps_medest,eps_meanest,eps_stdevest,eps_actual,prc,market_equity,div12m_me,chcsho_12m,eqnpo_12m,ret_1_0,ret_3_1,ret_6_1,ret_9_1,ret_12_1,ret_12_7,ret_60_12,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,at_gr1,sale_gr1,capx_gr1,inv_gr1,debt_gr3,sale_gr3,capx_gr3,inv_gr1a,lti_gr1a,sti_gr1a,coa_gr1a,col_gr1a,cowc_gr1a,ncoa_gr1a,ncol_gr1a,nncoa_gr1a,fnl_gr1a,nfna_gr1a,tax_gr1a,be_gr1a,ebit_sale,gp_at,cop_at,ope_be,ni_be,ebit_bev,netis_at,eqnetis_at,dbnetis_at,oaccruals_at,oaccruals_ni,taccruals_at,taccruals_ni,noa_at,opex_at,at_turnover,sale_bev,rd_sale,cash_at,sale_emp_gr1,emp_gr1,ni_inc8q,noa_gr1a,ppeinv_gr1a,lnoa_gr1a,capx_gr2,saleq_gr1,niq_be,niq_at,niq_be_chg1,niq_at_chg1,rd5_at,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,saleq_su,niq_su,capex_abn,op_atl1,gp_atl1,ope_bel1,cop_atl1,pi_nix,ocf_at,op_at,ocf_at_chg1,at_be,ocfq_saleq_std,tangibility,earnings_variability,aliq_at,f_score,o_score,z_score,intrinsic_value,kz_index,ni_ar1,ni_ivol,at_me,be_me,debt_me,netdebt_me,sale_me,ni_me,ocf_me,fcf_me,eqpo_me,eqnpo_me,rd_me,bev_mev,ebitda_mev,aliq_mat,eq_dur,beta_60m,resff3_12_1,resff3_6_1,mispricing_mgmt,mispricing_perf,ivol_capm_21d,iskew_capm_21d,coskew_21d,beta_dimson_21d,ivol_ff3_21d,iskew_ff3_21d,ivol_hxz4_21d,iskew_hxz4_21d,rmax5_21d,rmax1_21d,rvol_21d,rskew_21d,zero_trades_21d,dolvol_126d,dolvol_var_126d,turnover_126d,turnover_var_126d,zero_trades_126d,zero_trades_252d,ami_126d,ivol_capm_252d,prc_highprc_252d,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
0,0,2000-01-31,20000131,10001,11,3,0.031007,2000,1,0.0041,-0.048218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.50000,20.825000,0.055722,0.011979,0.043875,-0.004188,0.082031,0.018678,0.018095,-0.062294,-0.085540,0.462718,0.000000,-0.005199,0.000905,0.009279,0.019392,0.241431,0.237890,-0.607539,-0.016962,0.707037,-0.187323,-0.073388,-0.000185,0.000000,-0.024617,0.044149,-0.068765,0.039942,1.271294e-03,0.038670,-0.043062,0.042877,0.004831,0.021658,0.060605,0.114070,0.222407,0.196271,0.090494,0.105962,-0.042877,0.000023,-0.042900,-0.108337,-2.953371,-0.065460,-1.784499,0.716046,1.121651,1.247588,1.748406,NaN,0.005224,0.183239,0.048000,1.0,-0.030679,0.001791,0.021610,0.163393,0.373990,0.017047,0.005001,0.021390,0.006267,NaN,0.961262,-0.210942,-0.363526,NaN,0.985859,1.879607,-0.303273,0.116282,0.116282,0.207349,0.226720,1.672968,0.145020,0.114070,0.028173,2.466956,0.058552,0.748893,0.100553,0.578352,6.0,-0.625089,2.465921,13.496698,0.242402,0.382203,0.005754,2.077455,0.842113,0.829340,0.818487,2.567155,0.076206,0.301273,0.122113,0.054982,0.054934,NaN,0.807420,0.130314,0.499475,15.189621,0.469139,-0.157213,0.243031,0.637786,0.408565,0.022738,-0.030785,-0.381627,1.361329,0.022407,-0.592304,0.023703,-0.354362,0.028884,0.050882,0.022777,0.032045,3.827754,10314.692627,1.443015,0.000484,1.479803,3.126324,3.092420,5.038418,0.027127,0.800000,0.598605,0.007935,0.040985,0.100657,1.064359,168,-0.320694,0.362350,-0.211910,-0.430240
1,1,2000-01-31,20000131,10002,11,3,0.014570,2000,1,0.0041,-0.029741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.18750,94.197187,0.029128,0.000000,0.026215,-0.123036,-0.060797,0.048151,0.031190,-0.064491,-0.129900,0.950592,0.016393,-0.016041,-0.003827,0.017878,0.221328,0.257762,NaN,3.269231,3.510034,1.101568,NaN,0.000174,0.058091,0.030309,NaN,NaN,NaN,NaN,NaN,NaN,0.015208,-0.015208,0.001149,0.017976,0.274889,0.052080,NaN,NaN,0.110537,0.917319,NaN,NaN,0.015208,NaN,NaN,NaN,NaN,NaN,0.062711,0.099301,0.829841,NaN,0.039467,NaN,NaN,8.0,NaN,NaN,0.004556,NaN,-0.040042,0.027942,0.003282,-0.007274,-0.001050,NaN,-2.023276,-0.078877,-0.063142,0.022967,-2.547327,-2.160420,NaN,0.033699,0.063607,NaN,NaN,1.379267,0.013992,0.027592,-0.002488,8.441148,0.028131,NaN,0.675236,NaN,NaN,NaN,NaN,52.111766,-25.127401,-0.579022,0.001106,5.200601,0.616101,0.155079,-0.050171,0.469632,0.068102,0.074843,NaN,NaN,NaN,NaN,0.595823,0.151077,NaN,15.923299,0.237320,-0